In [1]:
#chat_id = YOUR_CHAT_ID
api_id = 329388
api_hash = '5f7f7f23f9407bd80f33380c61e4c1a0'

In [3]:
from telethon import TelegramClient, sync

import socks

host = '52.178.155.48'  # a valid host
port = 31415  # a valid port
username = 'yeahrmek'
password = '1qaz@WSX'

client = TelegramClient('session_name', api_id, api_hash,
                        proxy=(socks.SOCKS5, host, port, True, username, password))
client.start()

In [4]:
print(client.get_me().stringify())

User(
	id=75343074,
	is_self=True,
	contact=True,
	mutual_contact=True,
	deleted=False,
	bot=False,
	bot_chat_history=False,
	bot_nochats=False,
	verified=False,
	restricted=False,
	min=False,
	bot_inline_geo=False,
	access_hash=584916513175492651,
	first_name='Yermek',
	last_name=None,
	username='yeahrmek',
	phone='79166354525',
	photo=UserProfilePhoto(
		photo_id=323596039266347028,
		photo_small=FileLocation(
			dc_id=2,
			volume_id=239311079,
			local_id=175777,
			secret=2990242154546046445
		),
		photo_big=FileLocation(
			dc_id=2,
			volume_id=239311079,
			local_id=175779,
			secret=-666989345791161627
		)
	),
	status=UserStatusOffline(
		was_online=datetime.datetime(2018, 9, 21, 11, 3, 21, tzinfo=datetime.timezone.utc)
	),
	bot_info_version=None,
	restriction_reason=None,
	bot_inline_placeholder=None,
	lang_code=None
)


In [25]:
phystech_party = None
for d in dialogs:
    if d.name == 'Физтех-пати':
        phystech_party = d
        break

Get chat history

In [47]:
from telethon.tl.types import PeerUser, PeerChat, PeerChannel

channel = PeerChannel(phystech_party.id)
phystech_party_chat = client.get_input_entity(channel)

In [64]:
users = dict()

from tqdm import tqdm

for message in tqdm(client.iter_messages(phystech_party_chat)):
    sender = message.sender
    username = message.sender.username
    if not username:
        username = message.sender.first_name
        
        
    if not username in users:
        users[username] = dict(
            dates=[],
            messages=[]
        )
    users[username]['dates'].append(message.date)
    users[username]['messages'].append(message.text)

7036it [01:10, 100.12it/s]


In [66]:
def count_messages(users, username):
    info = users[username]
    n_messages = len(info['messages'])
    return n_messages

In [68]:
for username in users:
    users[username]['n_messages'] = count_messages(users, username)

In [74]:
def key(x):
    print(x)
sorted_users = sorted(users, key=lambda x: -users[x]['n_messages'])

In [79]:
for u, n in zip(sorted_users, [users[u]['n_messages'] for u in sorted_users]):
    print('{}: {}'.format(u, n))

Mayorovea: 2137
yeahrmek: 847
Pasha: 798
junkie_style: 771
Дмитрий: 570
nikitasmagin: 568
SublimeBot: 539
Yakovets: 234
pnovichkov: 151
Павел: 147
Pavel: 111
Игорь Олегович: 73
Георгий: 33
gafrustam: 24
PollBot: 22
detsepe_bot: 5
fuckedGirlBot: 4
yetanotherstatsbot: 2


In [92]:
import pandas as pd

pd.to_datetime(users['yeahrmek']['dates'][-1])

Timestamp('2018-04-02 15:13:44+0000', tz='UTC')

In [94]:
bot_date = pd.to_datetime(users['PollBot']['dates'][-1])

In [96]:
from copy import deepcopy

subset_messages = deepcopy(users)
dates = dict.fromkeys(subset_messages.keys())
for user in subset_messages:
    dates[user] = pd.to_datetime(subset_messages[user]['dates'])

In [101]:
for user in dates:
    df = dates[user]
    dates[user] = dates[user][dates[user] > bot_date]

In [105]:
sorted_users = sorted(dates, key=lambda x: -len(dates[x]))

In [106]:
for user in sorted_users:
    print('{}: {}'.format(user, len(dates[user])))

Mayorovea: 915
SublimeBot: 410
junkie_style: 359
Pasha: 337
yeahrmek: 292
nikitasmagin: 285
Дмитрий: 180
Yakovets: 121
pnovichkov: 105
Pavel: 84
Павел: 76
Игорь Олегович: 37
Георгий: 24
PollBot: 20
detsepe_bot: 5
gafrustam: 2
yetanotherstatsbot: 2
fuckedGirlBot: 0
